In [349]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [350]:
# URL of the Wikipedia page
url_0_C = "https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(0%E2%80%939,_A%E2%80%93C)"
url_D_J = "https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(D%E2%80%93J)"
url_K_R = "https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(K%E2%80%93R)"
url_S_Z = "https://en.wikipedia.org/wiki/List_of_fiction_works_made_into_feature_films_(S%E2%80%93Z)"

urls = [url_0_C, url_D_J, url_K_R, url_S_Z]


In [351]:
def scrap_book_to_movie(url): 
    response = requests.get(url)
    result = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        content = soup.find('div', {'class': 'mw-parser-output'})
        tables = content.find_all('table', {'class': 'wikitable'})
        for table in tables:
            rows = table.find_all('tr')
            
            for row in rows:
                cells = row.find_all('td')

                # splits into book and movie
                cell_tab = [cell.get_text(strip=True) for cell in cells]
                result.append(cell_tab)    

    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

    result = pd.DataFrame(result)
    result.columns = ['fiction_work', 'film_adaptations']
    return result
    

In [352]:
library = scrap_book_to_movie(url)
library.loc[library['film_adaptations'].isnull() & library['fiction_work'].notnull(), ['film_adaptations']] = library['fiction_work']
library.loc[library['film_adaptations'] == library['fiction_work'], ['fiction_work']] = None


In [353]:
# fill nan fiction_work values with the last non null value of fiction_work
library['fiction_work'] = library['fiction_work'].ffill()
# drop nan where both columns are nan
library = library.dropna(subset=['film_adaptations'])

In [354]:
def extract_years(text):
    format = []
    #year = re.findall(r'\((?:.*)(\d{4}[–-]\d{4})(?:.*)\)|\((?:.*)(\d{4}[–-]present)(?:.*)\)|\((?:.*)(\d{4})(?:.*)\)', text)
    years = re.findall(r'\((?:[^)]*?)(\d{4}(?:[–-](?:\d{4}|present))?)(?:[^)]*?)\)', text)
    #print(year)
    #if year == []:
    #    print('hi')
    return years[0] if years else None

In [355]:
library['title_book'] = library['fiction_work'].str.split('(').str[0]
library['author_book'] = library['fiction_work'].str.split(',').str[-1]
library['year_book'] = library['fiction_work'].apply(extract_years)

library['title_film'] = library['film_adaptations'].str.split('(').str[0]
library['year_film'] = library['film_adaptations'].apply(extract_years)

library = library.drop(['fiction_work', 'film_adaptations'], axis = 1)
save =  library.copy()

In [356]:
library = save.copy()
indexes = library.index[library['title_film'] == 'same as above'].tolist()
#print(indexes)
target_ind =[(index - 1) for index in indexes]
library['title_film'][indexes] = library['title_film'][target_ind]
library['year_film'][indexes] = library['year_film'][target_ind]

# print(library.loc[indexes], library.loc[target_ind])

indexes = library.index[library['title_film'] == 'same as below'].tolist()
#print(indexes)
target_ind =[(index + 1) for index in indexes]
library['title_film'][indexes] = library['title_film'][target_ind]
library['year_film'][indexes] = library['year_film'][target_ind]

# print(library.loc[indexes], library.loc[target_ind])

C:\Users\Iris\AppData\Local\Temp\ipykernel_12664\3180876489.py:5: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  library['title_film'][indexes] = library['title_film'][target_ind]
C:\Users\Iris\AppData\Local\Temp\ipykernel_12664\3180876489.py

In [357]:
year_film = library.copy()['year_film']
print(library[year_film.isnull()])

year_book = library.copy()['year_book']
print(library[year_book.isnull()])



library.to_csv('table_D_J.csv')


Empty DataFrame
Columns: [title_book, author_book, year_book, title_film, year_film]
Index: []
      title_book        author_book year_book  title_film year_film
133  Dharamputra  Acharya Chatursen      None  Dharmputra      1961


In [ ]:
dataframes = []
for url in urls: 
    df = scrap_book_to_movie(url)
    clean_df = #TO COMPLETE
    dataframes.append(clean_df)

book_adaptations = pd.concat(dataframes)
book_adaptations.to_csv('book_adaptations.csv')